In [1]:
import itertools
import os
import shutil
import string

import numpy as np
import pandas as pd

In [2]:
class Problem(object):
    
    def __init__(self, problem_name, has_subtests=True):
        self.problem_name = problem_name
        self.test_id = 0
        self.has_subtests = has_subtests
        if os.path.exists(self.problem_name):
            shutil.rmtree(self.problem_name)
        os.makedirs(f'{self.problem_name}/input')
        os.makedirs(f'{self.problem_name}/output')
        
    def create_inputs(self, **kwargs):
        """Should return a list. Each element is a line in the input."""
        raise NotImplementedError
    
    def solve(self, inputs, **kwargs):
        """Should return a list. Each element is a line in the output."""
        raise NotImplementedError
    
    @staticmethod
    def save_to_file(data, file_path=None, **kwargs):
        if isinstance(data, list):
            data = ' '.join(str(x) for x in data)
        print(data, file=file_path, **kwargs)
    
    def create_single_test(self, inputs=None, **kwargs):
        n = 1
        if inputs is None:
            if self.has_subtests:
                if 'N' in kwargs:
                    n = kwargs['N']
                else:
                    n = np.random.randint(kwargs['N_MIN'], kwargs['N_MAX']+1)
                inputs = [n]
                outputs = []
                for _ in range(n):
                    inp = self.create_inputs(**kwargs)
                    outp = self.solve(inp)
                    inputs += inp
                    outputs += outp
            else:
                inputs = self.create_inputs(**kwargs)
                outputs = self.solve(inputs)
        else:
            outputs = self.solve(inputs, **kwargs)
            if self.has_subtests:
                inputs = [1] + inputs

        self.write_test(self.test_id, inputs, outputs)
        self.test_id += 1
        return (self.test_id, inputs, outputs)
    
    def create_multiple_tests(self, n, **kwargs):
        for i in range(n):
            yield self.create_single_test(**kwargs)
            
    def write_test(self, test_id, inputs, outputs):
        input_file = f'{self.problem_name}/input/input_{test_id:02d}.txt'
        output_file = f'{self.problem_name}/output/output_{test_id:02d}.txt'

        with open(input_file, 'w') as fp:
            for line in inputs:
                Problem.save_to_file(line, fp)

        with open(output_file, 'w') as fp:
            for line in outputs:
                Problem.save_to_file(line, fp)
    
    def create_from_topcoder(self, filepath, input_parser=None, output_parser=None, max_test=50, **kwargs):
        data = pd.read_csv(filepath, sep='\t', header=None).sample(frac=1)
        for i in range(min(len(data), max_test)):
            inputs = data.iloc[i, 0] if input_parser is None else input_parser(data.iloc[i, 0])
            outputs = data.iloc[i, 1] if output_parser is None else output_parser(data.iloc[i, 1])
            self.write_test(i, inputs, outputs)
            self.test_id += 1
            yield (i, inputs, outputs)
    

def parse_topcoder_data(data):
    data = data.replace('{', '[').replace('}', ']')
    data = eval(data)
    if not isinstance(data, tuple):
        data = [data]
    return data

In [43]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])

class CountRectangles(Problem):
    def __init__(self):
        super().__init__('count-rectangles', False)

    def create_inputs(self, n=10000, x_max=200, y_max=200, **kwargs):
        assert n <= x_max*y_max
        points = list(range(x_max*y_max))
        np.random.shuffle(points)
        return [n] + [f'{p%x_max} {p//x_max}' for p in points]
    
    def solve(self, inputs, **kwargs):
        n, *points = inputs
        points = [Point(*(int(x) for x in p.split(' '))) for p in points]

        hashset = set(points)
        count = 0
        for i, p1 in enumerate(points):
            for j in range(i+1, n):
                p2 = points[j]
                if p1.x == p2.x or p1.y == p2.y:
                    continue
                count += ((p1.x, p2.y) in hashset) and ((p2.x, p1.y) in hashset)
        
        return [count//2]

In [44]:
problem = CountRectangles()

In [47]:
problem.create_single_test([
    9,
    '0 1',
    '1 0',
    '1 1',
    '0 0',
    '2 2',
    '2 0',
    '0 2',
    '3 1',
    '3 0'
])

(3, [9, '0 1', '1 0', '1 1', '0 0', '2 2', '2 0', '0 2', '3 1', '3 0'], [4])